<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

-  Aperture photometry with local-background subtraction using CircularAnnulus apertures
  - Using the mean within a circular annulus
  - Using the sigma-clipped median within a circular annulus
</div>

# Local Background Subtraction

## Preliminaries

In [ ]:
# initial imports
import numpy as np
import matplotlib.pyplot as plt

# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'

hdr = fits.getheader(sci_fn)
data = fits.getdata(sci_fn)
error = fits.getdata(rms_fn)
wcs = WCS(hdr)

In [ ]:
# calculate the total error:  background error plus source Poisson error
from photutils.utils import calc_total_error

eff_gain = hdr['TEXPTIME']
total_error = calc_total_error(data, error, eff_gain)

The background in the XDF image has already been subtracted.  Let's add a background of `7.2 e-/s` (per pixel) so we'll have something to subtract in these examples.

In [ ]:
data += 7.2

In [ ]:
# display the data
from astropy.visualization import simple_norm

plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', percent=99.5)
plt.imshow(data, norm=norm)
cbar = plt.colorbar(shrink=0.8)

## Perform aperture photometry at multiple positions

In [ ]:
from photutils.aperture import CircularAperture

# define three apertures
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.0  # pixels
circ_aper = CircularAperture(positions, r=radius)

In [ ]:
# plot the apertures
plt.figure(figsize=(5, 5))
plt.imshow(data, norm=norm)
circ_aper.plot(color='red', lw=2);

In [ ]:
# perform aperture photometry for the three sources
# data here includes background, so the aperture sums are *not* the source fluxes
# ideally the data array should be background subtracted before performing aperture photometry
import astropy.units as u
from photutils.aperture import ApertureStats

unit = u.electron / u.s
circ_apstats = ApertureStats(data << unit, circ_aper, error=total_error << unit)
circ_apstats.to_table()

## Local background estimation

One often wants to also estimate the local background around each source using a nearby aperture or annulus aperture surrounding each source.  A simple method for doing this is to use the `ApertureStats` class to compute the mean background level within the background aperture. This class can also be used to calculate more advanced statistics (e.g., a sigma-clipped median) within the background aperture (e.g., a circular annulus). We show examples of both below.

Let's start by creating circular-annulus apertures at the same positions as the `CircularApertures` defined above. We'll define each circular annulus to have an inner and outer radius of 10 and 15 pixels, respectively.

The circular-annulus apertures will be used for local background estimation around the sources.

In [ ]:
from photutils.aperture import CircularAnnulus

bkg_aper = CircularAnnulus(positions, r_in=10.0, r_out=15.0)
apers = [circ_aper, bkg_aper]

In [ ]:
# plot the apertures
plt.figure(figsize=(5, 5))
plt.imshow(data, norm=norm)
aper_patches = circ_aper.plot(color='white', lw=2, label='Photometry aperture')
bkg_patches = bkg_aper.plot(color='orange', lw=2, label='Background annulus', hatch='///')
handles = (aper_patches[0], bkg_patches[0])
plt.legend(loc=(0.03, 0.85), facecolor='#458989', labelcolor='white',
           handles=handles, prop={'weight': 'bold', 'size': 11});

### Simple mean within a circular annulus

This first example uses the mean value in circular annulus as the background value.

We'll use the `ApertureStats` class to compute the mean background level (per pixel) within the annulus aperture at each position.

In [ ]:
from photutils.aperture import ApertureStats

bkg_apstats = ApertureStats(data << unit, bkg_aper)
bkg_mean = bkg_apstats.mean
print(bkg_mean)

The total background within the circular apertures is then the per-pixel background level multiplied by the circular-aperture areas.  The circular-aperture areas can be calculated from the `ApertureStats` object with a circular aperture (defined above).

In [ ]:
total_bkg = bkg_mean * circ_apstats.sum_aper_area.value
print(total_bkg)

Finally, let's calculate the local background-subtracted sum within the circular apertures.

In [ ]:
apersum_bkgsub = circ_apstats.sum - total_bkg
print(apersum_bkgsub)

### Sigma-clipped median within a circular annulus

For this example, the local background level around each source is estimated as the sigma-clipped median value within the circular annulus. We’ll use the `ApertureStats` class to compute both the photometry (aperture sum) and the background level.

We'll use the same circular and circular-annulus apertures defined above.

In [ ]:
from astropy.stats import SigmaClip

sigclip = SigmaClip(sigma=3.0, maxiters=10)
circ_stats = ApertureStats(data << unit, circ_aper, sigma_clip=None)
bkg_stats = ApertureStats(data << unit, bkg_aper, sigma_clip=sigclip)

Now, let's calculate the sigma-clipped median values in the background annulus apertures.

Note that the values are *very* close to the background value of `7.2` that we added to the data at the beginning of this notebook.

In [ ]:
bkg_stats.median

As before, the total background within the circular apertures is then the per-pixel background level multiplied by the circular-aperture areas.

In [ ]:
total_bkg = bkg_stats.median * circ_stats.sum_aper_area.value
print(total_bkg)

Finally, let's calculate the local background-subtracted sum within the circular apertures.

In [ ]:
apersum_bkgsub = circ_stats.sum - total_bkg
print(apersum_bkgsub)

Note that if you want to compute all of the source properties (i.e., in addition to only sum) on the local-background-subtracted data, you may input the per-pixel local background values to `ApertureStats` via the `local_bkg` keyword.

In [ ]:
aper_stats_bkgsub = ApertureStats(data << unit, circ_aper,
                                  local_bkg=bkg_stats.median << unit)
print(aper_stats_bkgsub.sum)

Note these background-subtracted values are the same as those above.

<div class="alert alert-warning alert-block">
<h3 style='margin-top: 0;'>Learn More</h3>

The [segmentation notebook](04-segmentation.ipynb) covers:
    
- The basics of image segmentation
</div>